In [59]:
import os
import tweepy
import pandas as pd

def get_tweets(hashtag):
    # Autenticación
    consumer_key = os.getenv("tw_consumer_key")
    consumer_secret = os.getenv("tw_consumer_secret")
    access_token = os.getenv("tw_access_token")
    access_token_secret = os.getenv("tw_access_token_secret")

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth)
    tweets_data = []
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, lang="es").items(50):
        tweet_data = {
            'text': tweet.text,
            'user': tweet.user.screen_name,
            'is_retweet': hasattr(tweet, 'retweeted_status')
        }
        tweets_data.append(tweet_data)
    df = pd.DataFrame(tweets_data, columns=['text', 'user', 'is_retweet'])
    return df

In [60]:
df = get_tweets("#inteligenciaartificial")

In [61]:
df.head()

,text,user,is_retweet
0,En un mundo donde la #InteligenciaArtificial e...,paisanxs,False
1,RT @conectapaciente: Un nuevo software que uti...,marermi13,True
2,🚀 ¡Maximiza tu experiencia con #ChatGPT al máx...,IAconNaudy,False
3,RT @adigital_org: 🧠En medio de la vorágine de ...,paisanxs,True
4,Feliz día a aquellos que se la saben todas per...,asocdepolo,False


In [62]:
## escribe una funcion para crear una nueva columna con el usuario oroginal del tweet sin tener en cuenta las letras RT, en caso que is_retweet sea True 

def get_original_user(row):
    if row['is_retweet']:
        return row['text'].split(':')[0]
    else:
        return '@'+row['user']

In [63]:
df['original_user'] = df.apply(get_original_user, axis=1)

In [64]:
df.head()

,text,user,is_retweet,original_user
0,En un mundo donde la #InteligenciaArtificial e...,paisanxs,False,@paisanxs
1,RT @conectapaciente: Un nuevo software que uti...,marermi13,True,RT @conectapaciente
2,🚀 ¡Maximiza tu experiencia con #ChatGPT al máx...,IAconNaudy,False,@IAconNaudy
3,RT @adigital_org: 🧠En medio de la vorágine de ...,paisanxs,True,RT @adigital_org
4,Feliz día a aquellos que se la saben todas per...,asocdepolo,False,@asocdepolo


In [65]:
## crea una funcion que elimine los espacios en blanco y las letras RT de la columna original_user en caso que el texto empiece con RT 

def clean_original_user(row):
    if row['original_user'].startswith('RT'):
        return row['original_user'][2:]
    else:
        return row['original_user']

In [66]:
df['original_user'] = df.apply(clean_original_user, axis=1)

In [67]:
df.head()

,text,user,is_retweet,original_user
0,En un mundo donde la #InteligenciaArtificial e...,paisanxs,False,@paisanxs
1,RT @conectapaciente: Un nuevo software que uti...,marermi13,True,@conectapaciente
2,🚀 ¡Maximiza tu experiencia con #ChatGPT al máx...,IAconNaudy,False,@IAconNaudy
3,RT @adigital_org: 🧠En medio de la vorágine de ...,paisanxs,True,@adigital_org
4,Feliz día a aquellos que se la saben todas per...,asocdepolo,False,@asocdepolo


In [68]:
## Crea una fuuncion para eliminar todos los emoji de la columna text

def remove_emoji(text):
    return text.encode('ascii', 'ignore').decode('ascii')

df['text'] = df['text'].apply(remove_emoji)    

In [69]:
df.head()

,text,user,is_retweet,original_user
0,En un mundo donde la #InteligenciaArtificial e...,paisanxs,False,@paisanxs
1,RT @conectapaciente: Un nuevo software que uti...,marermi13,True,@conectapaciente
2,Maximiza tu experiencia con #ChatGPT al mximo...,IAconNaudy,False,@IAconNaudy
3,RT @adigital_org: En medio de la vorgine de in...,paisanxs,True,@adigital_org
4,Feliz da a aquellos que se la saben todas pero...,asocdepolo,False,@asocdepolo
